In [1]:
# 采用 tf2.x 环境单卡 V100 bs=5096 2080ti bs=2048
# 网络采用tx比赛部分preLn transformer lstm 网络 seq mask 采用的V2
# dense 只用了step2的299特征 如果加上step3的特征会有1000多特征，可能能上一点点，比较鸡肋
# 可以调整的参数nn的dropout（0.1可以更小试试） dense dropput等 seq 选用的列，seq预训练的维度 conv1d压缩的维度 transformer 的hidden units等
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('precision', 5)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('max_colwidth', 200)
pd.set_option('display.width', 5000)
from itertools import chain
from base import Cache
import gc
import os
os.environ['TF_DETERMINISTIC_OPS'] = '1'
from tqdm import tqdm
from base.trans_layer import Add, LayerNormalization
from base.trans_layer import MultiHeadAttention, PositionWiseFeedForward
from base.trans_layer import PositionEncoding
import joblib
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow import keras
from tensorflow.keras import optimizers, layers, losses
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Concatenate, GlobalMaxPooling1D, Flatten
from tensorflow.keras.backend import concatenate
from gensim.models import Word2Vec, KeyedVectors
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.python.keras.layers import CuDNNLSTM
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr.feature_column import  SparseFeat, DenseFeat, get_feature_names, build_input_features, get_linear_logit, DEFAULT_GROUP_NAME, input_from_feature_columns
from deepctr.layers.core import PredictionLayer, DNN
from deepctr.layers.interaction import FM
from deepctr.layers.utils import concat_func, add_func, combined_dnn_input

import random
SEED = 999
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = '3'
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
from multiprocessing import Pool
print(tf.__version__)
data = Cache.reload_cache('CACHE_data_step_2_feature_0917_r5.pkl')# 基础特征 0.797+
# 重置index唯一值
del data['index']
del data['communication_onlinerate']
gc.collect()
data = data.reset_index(drop=True).reset_index()
## window特征+2k

last_seq_list = ['creat_type_cd','tags','spread_app_id','task_id','adv_id','label']

def get_emb_matrix(col):
    """
    inputs:    
    col 需要做成预训练emb_matrix的列
    
    加载：
    emb_dict 预训练的词向量
    word_emb_dict 字典
    id_list_dict 字典索引序列
    
    得出id_list_dict+emb_matrix
    """
    id_list_dict_all = Cache.reload_cache(f'CACHE_EMB_INPUTSEQ_V2_{col}.pkl')
#     id_list_dict = id_list_dict_all['id_list']
#     key2index = id_list_dict_all['key2index']
#     emb = id_list_dict_all['emb']
    key_to_represent_rare = '-1'
    words = list(id_list_dict_all['emb'].keys())
    emb_size = id_list_dict_all['emb'][words[0]].shape[0]
    voc_size = len(words)
    emb_matrix = np.zeros((voc_size + 1, emb_size))
    # emb 中必须要有'-1' 作为index 0
    if '-1' not in id_list_dict_all['key2index'].keys():
        #  emb中无-1 为全词表数据！需要自行计算均值emb vec
        # 为embi 添加一个embedding
        # 这些词的vector求均值
        vector_low_frequency_words = np.zeros((emb_size,))
        for w in words:
            vector_low_frequency_words += id_list_dict_all['emb'][w]
        vector_low_frequency_words = vector_low_frequency_words / voc_size
        # emb添加一个key value
        id_list_dict_all['emb'][key_to_represent_rare] = vector_low_frequency_words
        # print(f'{col} file has no key_to_represent_rare add low frequency words and fill vector as:', vector_low_frequency_words)
    for k, idx in id_list_dict_all['key2index'].items():
        try:
            emb_matrix[idx, :] = id_list_dict_all['emb'][k]
        except KeyError:  # 如果k不在不在word_emb_dict中，则默认用max_key_to_represent_rare填充
            #                 print('find oov:',(k, idx))
            emb_matrix[idx, :] = id_list_dict_all['emb'][key_to_represent_rare]
    emb_matrix = np.float32(emb_matrix)
    return {col:[id_list_dict_all['id_list'],emb_matrix]}


with Pool(3) as p:
    res = p.map(get_emb_matrix, last_seq_list)
id_list_dict_emb_all = {}
for item in res:
    id_list_dict_emb_all.update(item)
del res,item
gc.collect()

GlobalSeqLength = 40
base_inputdim_dict = {}
for var in id_list_dict_emb_all.keys():
    base_inputdim_dict[var] = id_list_dict_emb_all[var][1].shape[0]
base_embdim_dict = {'creat_type_cd':32,'tags':32,'spread_app_id':32,'task_id':32,'adv_id':32,'label':32}
conv1d_info_dict = {'creat_type_cd':8,'tags':8,'spread_app_id':8,'task_id':16,'adv_id':16,'label':8}
TRAINABLE_DICT = {'creat_type_cd':False,'tags':False,'spread_app_id':False,'task_id':False,'adv_id':False,'label':False}
arr_name_list = list(id_list_dict_emb_all.keys())

def get_seq_input_layers(cols):
    print("Prepare input layer:", cols)
    inputs_dict = {}
    for col in cols:
        inputs_dict[col] = tf.keras.Input(shape=(GlobalSeqLength, ),
                                          dtype="int32",
                                          name=col+'_seq_layer')
    return inputs_dict

def get_emb_layer(col, emb_matrix=None, seq_length=None, trainable=False):
    if seq_length is None:
        seq_length = GlobalSeqLength
    if trainable==True:
        emb_layer = tf.keras.layers.Embedding(base_inputdim_dict[col],
                                              base_embdim_dict[col],
                                              input_length=seq_length,
                                              dtype="float32",
                                              trainable=True)
    else:
        embedding_dim = emb_matrix.shape[-1]
        input_dim = emb_matrix.shape[0]
        emb_layer = tf.keras.layers.Embedding(input_dim,
                                              embedding_dim,
                                              input_length=seq_length,
                                              weights=[emb_matrix],
                                              dtype="float32",
                                              trainable=trainable)
    return emb_layer

def trans_net(inputs,masks ,hidden_unit=128):
    inputs = tf.keras.layers.Dropout(0.3)(inputs)
    encodings = tf.keras.layers.Conv1D(filters=inputs.shape[-1], kernel_size=1, padding='same', activation='relu')(inputs)
    # trans tunnel
    for i in range(1):
        # pre Norm
        encodings = LayerNormalization()(encodings)
        # Masked-Multi-head-Attention
        masked_attention_out = MultiHeadAttention(8, encodings.shape[-1] // 8)([encodings, encodings, encodings, masks])
        # Add & Norm
        masked_attention_out = masked_attention_out+ encodings
        # Feed-Forward
        ff = PositionWiseFeedForward(encodings.shape[-1], hidden_unit)
        ff_out = ff(masked_attention_out)
    # LSTM
    x = tf.keras.layers.Bidirectional(CuDNNLSTM(hidden_unit, return_sequences=True))(encodings)
    # linear
    x = tf.keras.layers.Conv1D(
        filters=encodings.shape[-1], kernel_size=1, padding='same', activation='relu')(x)
    # 3 项Add & Norm
    x = x + masked_attention_out + ff_out
    x = LayerNormalization()(x)
    return x


def create_model(linear_feature_columns, dnn_feature_columns, fm_group=[DEFAULT_GROUP_NAME], dnn_hidden_units=(128, 128),
                 l2_reg_linear=0.00001, l2_reg_embedding=0.00001, l2_reg_dnn=0, seed=1024, dnn_dropout=0,
                 dnn_activation='relu', dnn_use_bn=False, task='binary'):

    K.clear_session()
#!################################################################################################################
    seq_inputs_dict = get_seq_input_layers(cols=arr_name_list)
    inputs_all = list(seq_inputs_dict.values())  # 输入层list
    masks = tf.equal(seq_inputs_dict['task_id'], 0)
    # 普通序列+label序列
    layers2concat = []
    for index, col in enumerate(arr_name_list):
        print(col, 'get embedding!')
        emb_layer = get_emb_layer(col, trainable=TRAINABLE_DICT[col],emb_matrix=id_list_dict_emb_all[col][1])
        x = emb_layer(seq_inputs_dict[col])
        if conv1d_info_dict[col] > -1:
            cov_layer = tf.keras.layers.Conv1D(filters=conv1d_info_dict[col],
                                               kernel_size=1,
                                               activation='relu')
            x = cov_layer(x)
        layers2concat.append(x)
    x = keras.layers.concatenate(layers2concat)
#!################################################################################################################
#!mix1
    x = trans_net(x, masks, hidden_unit=256)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()
    average_pool = tf.keras.layers.GlobalAveragePooling1D()
    xmaxpool = max_pool(x)
    xmeanpool = average_pool(x)

    trans_output = tf.keras.layers.concatenate([xmaxpool, xmeanpool])


#!################################################################################################################
#!mix2
    features = build_input_features(
        linear_feature_columns + dnn_feature_columns)

    inputs_list = list(features.values())

    linear_logit = get_linear_logit(features, linear_feature_columns, seed=seed, prefix='linear',
                                    l2_reg=l2_reg_linear)

    group_embedding_dict, dense_value_list = input_from_feature_columns(features, dnn_feature_columns, l2_reg_embedding,
                                                                        seed, support_group=True)

    fm_logit = add_func([FM()(concat_func(v, axis=1))
                         for k, v in group_embedding_dict.items() if k in fm_group])

    dnn_input = combined_dnn_input(list(chain.from_iterable(
        group_embedding_dict.values())), dense_value_list)

    mix = concatenate([trans_output, dnn_input], axis=-1)  # !#mix

    dnn_output = DNN(dnn_hidden_units, dnn_activation, l2_reg_dnn, dnn_dropout,
                     dnn_use_bn, seed)(mix)

    dnn_logit = tf.keras.layers.Dense(
        1, use_bias=False, activation=None)(dnn_output)

    final_logit = add_func([linear_logit, fm_logit, dnn_logit])
    output = PredictionLayer(task)(final_logit)

#!################################################################################################################

    model = Model(inputs=inputs_all+[features],
                  outputs=[output])
    print(model.summary())
    return model

def multi_category_focal_loss2(gamma=2., alpha=.25):
    """
    Usage:
     model.compile(loss=[multi_category_focal_loss2(
         alpha=0.35, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    epsilon = 1.e-7
    gamma = float(gamma)
    alpha = tf.constant(alpha, dtype=tf.float32)

    def multi_category_focal_loss2_fixed(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)

        alpha_t = y_true * alpha + \
            (tf.ones_like(y_true) - y_true) * (1 - alpha)
        y_t = tf.multiply(y_true, y_pred) + tf.multiply(1 - y_true, 1 - y_pred)
        ce = -tf.math.log(y_t)
        weight = tf.pow(tf.subtract(1., y_t), gamma)
        fl = tf.multiply(tf.multiply(weight, ce), alpha_t)
        loss = tf.reduce_mean(fl)
        return loss

    return multi_category_focal_loss2_fixed

class DataGenerator(tf.keras.utils.Sequence):
    """Generates data for Keras"""

    def __init__(self,id_list_dict, df, feature_names, list_IDs,idtrain, batch_size=512, shuffle=True):
        self.id_list_dict = id_list_dict# seq data to split
        self.df = df# data_to split
        self.feature_names = feature_names
        self.batch_size = batch_size# bs actually use
        self.list_IDs = list_IDs# index range(all samples should train)
        self.idtrain = idtrain# train data from id_list_dict
        self.shuffle = shuffle# id true shuffle samples 
        self.on_epoch_end()
    
    def __len__(self):
        """Denotes the number of batches per epoch"""
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        """Generate one batch of data"""
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        X, Y = self.__data_generation(list_IDs_temp)#bs
        # X{name:df[name] for name in feature_names} y label
        return X, Y

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def __data_generation(self, list_IDs_temp):
        """Generates data containing batch_size samples"""
        X = {}
        for key in self.id_list_dict.keys():
            X[key+'_seq_layer'] = self.id_list_dict[key][0][self.idtrain][list_IDs_temp]# id_list_dict[key][0]为seq 
        for key in self.feature_names:
            X[key] = self.df[key].iloc[list_IDs_temp]
        Y = self.df['label'].iloc[list_IDs_temp]
        return X, Y
    
# 输入列
sparse_features=['task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id', 'dev_id', 'inter_type_cd', 'slot_id', 'spread_app_id', 'tags', 'app_first_class', 'app_second_class', 'city', 'device_name', 'career', 'gender', 'net_type', 'residence', 'emui_dev', 'indu_name', 'cmr_0', 'cmr_1', 'cmr_2', 'cmr_3', 'cmr_4', 'cmr_5', 'cmr_6', 'cmr_7', 'cmr_8', 'cmr_9', 'cmr_10', 'cmr_11', 'cmr_12', 'cmr_13', 'cmr_14', 'cmr_15', 'cmr_16', 'cmr_17', 'cmr_18', 'cmr_19', 'cmr_20', 'cmr_21', 'cmr_22', 'cmr_23', 'age', 'city_rank']
dense_features=[i for i in data.columns if i not in sparse_features+['index','id','uid','level_0','pt_d','label']]
# dense_features = dense_features+add_fe
print('sparse_features:')
print(sparse_features)
print('dense_features:')
print(dense_features)

# 特征处理
# Label Encoding for sparse features,and do simple Transformation for dense features
for feat in tqdm(sparse_features):
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
for feat in tqdm(dense_features):
    mms = MinMaxScaler(feature_range=(0, 1))
    data[feat] = mms.fit_transform(data[feat].values.reshape(-1,1)).astype(np.float32)
droplist = []
for var in tqdm(sparse_features+dense_features):
    if data[var].nunique()<2 or data[var].count()<2:
        droplist.append(var)
for var in droplist:
    dense_features.remove(var)
    del data[var]
print('find droplist:',droplist)
gc.collect()
#!################################################################################################################
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique(),embedding_dim=8)
                       for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1,)
                      for feat in dense_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
print('feature_names finish!')

# callback
lr_list = [0.001, 0.001, 0.001, 0.0005, 0.00025,
           0.000125, 6.25e-05, 3.125e-05, 2e-05, 2e-05, 2e-05]
def scheduler(epoch):
    if epoch < len(lr_list):
        return lr_list[epoch]
    else:
        return 2.5e-6
def get_callbacks(if_valid=True):
    '''
    :param count:
    :return:
    '''
    checkpoint_dir = 'models'
    checkpoint_prefix = os.path.join(
        checkpoint_dir, f"ckpt_zlhnn_model0920_v2_{if_valid}")

    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                                             save_weights_only=True,
                                                             monitor='val_AUC',
                                                             verbose=1,
                                                             save_freq="epoch",
                                                             save_best_only=True,
                                                             mode='max')
    reduce_lr_callback_trainall = tf.keras.callbacks.LearningRateScheduler(
        scheduler)

    earlystop_callback = tf.keras.callbacks.EarlyStopping(
        monitor="val_AUC",
        min_delta=0.00001,
        patience=3,
        verbose=1,
        mode="max",
        baseline=None,
        restore_best_weights=True,
    )
    csv_log_callback = tf.keras.callbacks.CSVLogger(
        filename='./logs/model_zlhnn_model0920_v2.log', separator=",", append=True)

    reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_AUC',
                                                              factor=0.25,
                                                              patience=1,
                                                              min_delta=2e-4,
                                                              min_lr=2e-5)
    if if_valid:
        callbacks = [checkpoint_callback, csv_log_callback,
                     earlystop_callback, reduce_lr_callback]
    else:
        callbacks = [checkpoint_callback,
                     csv_log_callback,  reduce_lr_callback_trainall]
    return callbacks

# compile model
# del model
# gc.collect()
print(' model compile start ……')
try:
    del model
    gc.collect()
    K.clear_session()
except:
    pass
model = create_model(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                         dnn_hidden_units=(512, 256, 256),dnn_dropout=0.1,task='binary')
model.compile(
optimizer=tf.keras.optimizers.Adam(lr=1e-3), loss=multi_category_focal_loss2(alpha=0.35),
metrics=['AUC'])
print(' model compile finish ……')

bs = 5096
count = 0
random_state = 1111
set1 = data.query('pt_d<8').copy()
set2 = data.query('pt_d==8').copy()
# del data
gc.collect()
idtrain=list(set1['index'])
idtest=list(set2['index'])
skf = StratifiedKFold(n_splits=10, random_state=random_state, shuffle=True)# 抽90% 训练
for i, (train_index, test_index) in enumerate(skf.split(set1, set1['label'])):
    print("FOLD | ", count+1)
    print("###"*35)
    gc.collect()

    # 模型输入
    # 训练集
    online_train_model_input = {}
    for var in id_list_dict_emb_all.keys():
        online_train_model_input[var+'_seq_layer']= id_list_dict_emb_all[var][0][idtrain][train_index]
    online_train_model_input.update({name: set1[name].values[train_index] for name in tqdm(feature_names)})
    y_true_train = set1['label'].values[train_index]
    print('train input built!')
    # 验证集
    online_valid_model_input = {}
    for var in id_list_dict_emb_all.keys():
        online_valid_model_input[var+'_seq_layer']= id_list_dict_emb_all[var][0][idtrain][test_index]
    online_valid_model_input.update({name: set1[name].values[test_index] for name in tqdm(feature_names)})
    y_true_valid = set1['label'].values[test_index]
    print('valid input built!')
    # 测试集
    online_test_model_input = {}
    for var in id_list_dict_emb_all.keys():
        online_test_model_input[var+'_seq_layer']= id_list_dict_emb_all[var][0][idtest]
    online_test_model_input.update({name: set2[name].values for name in tqdm(feature_names)})
    print('test input built!')
    callbacks = get_callbacks()
    hist = model.fit(online_train_model_input,y_true_train,
            epochs=40, 
            batch_size=bs,
            verbose=1, 
            callbacks=callbacks,
            validation_data=(online_valid_model_input, y_true_valid))
    print(hist.history)
    if count ==0:
        break
# 预测
y_pre = model.predict(
    online_test_model_input, verbose=2, batch_size=1024)
res = set2[['id']]
res['probability'] = y_pre
res = res.sort_values('id')
res.to_csv('./subs/submission_nn_0920V2_5096.csv', index=False)

2.0.0


[2020-09-20 13:29:08] - utils.py[line:41] - WARNING: 
DeepCTR version 0.8.1 detected. Your version is 0.8.0.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.8.1
[2020-09-20 13:29:19] - __init__.py[line:127] - INFO: Successfully Reload: /home/tione/notebook/huawei/cached_data/CACHE_data_step_2_feature_0917_r5.pkl
[2020-09-20 13:29:33] - __init__.py[line:127] - INFO: Successfully Reload: /home/tione/notebook/huawei/cached_data/CACHE_EMB_INPUTSEQ_V2_tags.pkl
[2020-09-20 13:29:33] - __init__.py[line:127] - INFO: Successfully Reload: /home/tione/notebook/huawei/cached_data/CACHE_EMB_INPUTSEQ_V2_creat_type_cd.pkl[2020-09-20 13:29:33] - __init__.py[line:127] - INFO: Successfully Reload: /home/tione/notebook/huawei/cached_data/CACHE_EMB_INPUTSEQ_V2_spread_app_id.pkl

[2020-09-20 13:29:40] - __init__.py[line:127] - INFO: Successfully Reload: /home/tione/notebook/huawei/cached_data/CACHE_EMB_INPUTSEQ_V2_task_id.pkl
[2020-09-20 13:29:43] -

sparse_features:
['task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id', 'dev_id', 'inter_type_cd', 'slot_id', 'spread_app_id', 'tags', 'app_first_class', 'app_second_class', 'city', 'device_name', 'career', 'gender', 'net_type', 'residence', 'emui_dev', 'indu_name', 'cmr_0', 'cmr_1', 'cmr_2', 'cmr_3', 'cmr_4', 'cmr_5', 'cmr_6', 'cmr_7', 'cmr_8', 'cmr_9', 'cmr_10', 'cmr_11', 'cmr_12', 'cmr_13', 'cmr_14', 'cmr_15', 'cmr_16', 'cmr_17', 'cmr_18', 'cmr_19', 'cmr_20', 'cmr_21', 'cmr_22', 'cmr_23', 'age', 'city_rank']
dense_features:
['device_size', 'his_app_size', 'his_on_shelf_time', 'app_score', 'list_time', 'device_price', 'up_life_duration', 'up_membership_grade', 'membership_life_duration', 'consume_purchase', 'communication_avgonline_30d', 'cmr_None', 'task_id_count', 'task_id_pt_d_count', 'adv_id_count', 'adv_id_pt_d_count', 'creat_type_cd_count', 'creat_type_cd_pt_d_count', 'adv_prim_id_count', 'adv_prim_id_pt_d_count', 'dev_id_count', 'dev_id_pt_d_count', 'inter_type_cd_count', 'inte

100%|██████████| 299/299 [00:26<00:00, 11.30it/s]


find droplist: []
feature_names finish!
 model compile start ……
Prepare input layer: ['creat_type_cd', 'tags', 'spread_app_id', 'task_id', 'adv_id', 'label']
creat_type_cd get embedding!
tags get embedding!
spread_app_id get embedding!
task_id get embedding!
adv_id get embedding!
label get embedding!
64 8 8
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
creat_type_cd_seq_layer (InputL [(None, 40)]         0                                            
__________________________________________________________________________________________________
tags_seq_layer (InputLayer)     [(None, 40)]         0                                            
__________________________________________________________________________________________________
spread_app_id_seq_layer (InputL [(None, 40)]         0                            

...

100%|██████████| 299/299 [00:00<00:00, 32202.57it/s]


test input built!
Train on 6841168 samples, validate on 760130 samples
Epoch 1/40


/opt/conda/envs/tensorflow2_py3/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/envs/tensorflow2_py3/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


6838832/6841168 [============================>.] - ETA: 0s - loss: 0.0411 - AUC: 0.7975
Epoch 00001: val_AUC improved from -inf to 0.81972, saving model to models/ckpt_zlhnn_model0920_v2_True
6841168/6841168 [==============================] - 311s 45us/sample - loss: 0.0411 - AUC: 0.7975 - val_loss: 0.0392 - val_AUC: 0.8197
Epoch 2/40
6838832/6841168 [============================>.] - ETA: 0s - loss: 0.0392 - AUC: 0.8192
Epoch 00002: val_AUC improved from 0.81972 to 0.82838, saving model to models/ckpt_zlhnn_model0920_v2_True
6841168/6841168 [==============================] - 288s 42us/sample - loss: 0.0392 - AUC: 0.8192 - val_loss: 0.0386 - val_AUC: 0.8284
Epoch 3/40
6838832/6841168 [============================>.] - ETA: 0s - loss: 0.0386 - AUC: 0.8268
Epoch 00003: val_AUC improved from 0.82838 to 0.83026, saving model to models/ckpt_zlhnn_model0920_v2_True
6841168/6841168 [==============================] - 288s 42us/sample - loss: 0.0386 - AUC: 0.8268 - val_loss: 0.0384 - val_AUC: 0

/opt/conda/envs/tensorflow2_py3/lib/python3.6/site-packages/ipykernel/__main__.py:469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [3]:
res.head()

,id,probability
3009371,1,0.21219
3788191,2,0.45186
1091887,3,0.41570
6693962,4,0.34192
3086987,5,0.35529
